# Log Analysis

Vault offers two types of logs:

- server logs
- audit logs

Server logs provide information on the software operation of Vault. This includes information such as leader elections, and other operational insights into what Vault is doing internally. The audit logs on the other hand provide logs on every request and response made to Vault. These are critical to understand the usage of Vault.

### Server Logs

Server logs are configured statically in the Vault configuration and offers quite [a few options](https://developer.hashicorp.com/vault/docs/configuration#log_level) including the rotation of log files. We will not look at this in depth, as there is little operational tasks that are Vault specific here. Typically these logs are written to file and ideally pushed/pulled to a log aggregation system such as Splunk.

### Audit Logs

Audit logs are configured via [the API](https://developer.hashicorp.com/vault/api-docs/system/audit) and support various audit deviced. **Audit logs are not enabled by default** and should thus be configured early on. Vault supports various audit devices such as:

- file (including stdout)
- syslog
- socket

In production, the best option is probably to use a socket device where a log forwarding daemon listens and forwards them to a central instance. A great article on the pros and cons of how to configure audit logging can be found [on the HashiCorp blog](https://www.hashicorp.com/en/blog/vault-logging-and-alerting-on-day-1).

In this tutorial we will enable audit logging and then try to analyse these logs.

### Starting Vault

Let us start a Vault server. This will run Vault in the background and push the logs to `/tmp/vault.log`. If at any point in time the Vault crashes, this command will need to be used again to re-launch the Vault server.

In [ ]:
nohup bash -c '
  vault server -dev -dev-root-token-id=root-token
' > /tmp/vault.log 2>&1 &
echo $! > /tmp/vault.pid

### Login to Vault

Let us login to Vault. We started a development version vor simplicity.

In [ ]:
export VAULT_ADDR="http://127.0.0.1:8200"
vault login root-token

### Enabling the Audit Logging

The audit logging can be enabled using the CLI with the `vault audit enable` subcommand. Have a look at [the command description](https://developer.hashicorp.com/vault/docs/commands/audit/enable) and enable audit logging to the `/tmp/vault-audit.log` file.

In [ ]:
vault audit enable ...

### Validate

The audit log should directly log that audit logging was enabled. Vault performs a "test audit" directly after enabling the audit log in order to ensure it can perform an audit log. This is critically important, as Vault will not perform requests while it cannot audit log (called log-request failures). In such cases Vault will shut down in order to avoid performing operations that are missing in the audit log.

Let us check that the audit log already contains such a test entry:

In [ ]:
# pretty print the first line in the audit log
head -n1 /tmp/vault-audit.log | jq

### Checking Audit Logs

As one can see, audit logs are formatted in JSON for easier parsing. In this first entry, one can see that the log is of type `"request"`, meaning an incoming call to Vault. This is opposed to `"response"` logs where Vault logs what it replied. Of course, the audit log contains a timestamp. In the case of requests, it contains information on what endpoint was called with what operation (in this case the `sys/audit/test` endpoint with an `update` - `PUT` operation). Let us have a look at the response:

In [ ]:
# pretty print the second line in the audit log
head -n2 /tmp/vault-audit.log | tail -n1 | jq

### Analysing Responses

As one can see, `"response"` type audit logs typically contain much more information. They typically contain a request part in order to link the response to the request that triggered the response. In this case, one can see that the response is **not** for the test call we saw in the first line of the log, but rather from the actual call creating the audit device (as seen by the `request.path` value that is `/sys/audit/file` - the endpoint we called with `vault audit enable` above).

Take a deeper look at the remaining information available in the audit log.

Note all the fields that contain `hmac-sha256:` prefixed in the values. This is because Vault handles a lot of sensitive data that should not land in any logs. Thus Vault hashes most response values before logging them. This hash **is unique to the Vault cluster and audit device** that generated it. This is important to avoid attacker trying to audit hash values on another Vault instance and compare the hashes to the ones present in a leaked audit log.

But then how do we actually find anything in the audit log? Vault offers an endpoint to audit-hash inputs, which can then be compared to the audit logs of that same Vault.

Let us first create a couple more entries in the audit log:

In [ ]:
. ./assets/populate-vault-secrets.sh

### Filtering the Audit Log

As the audit log is in JSON format, we can use `jq` to perform quite powerful searches in the audit log. For instance, to filter for audits on the path `teams/cloud-operations/app/backend/database` we can preform the following filter:

In [ ]:
jq 'select(.request.path == "secret/data/teams/cloud-operations/app/backend/database")' /tmp/vault-audit.log

### Finding Requests of Admin Users

Let us try to filter the audit log for requests that retrieved credentials with username `admin`. First, we will need to audit hash `admin`. Fill in the blank below to generate such a hash:

In [ ]:
export ADMIN_AUDIT_HASH="$(vault write -field=hash /sys/audit-hash/file input='...')"
echo $ADMIN_AUDIT_HASH

If you are unfamiliar with `jq`, you can find information on how to create queries and even an online playground here: https://jqlang.org/manual/#invoking-jq

Now let us filter the log entries that contained `username == admin` in the **response** (i.e. someone retrieved a credentials with `admin` as the username):

In [ ]:
jq "select(.response.data.data.username == \"$ADMIN_AUDIT_HASH\")" /tmp/vault-audit.log

### Your Turn

Ok, now it is your turn. The credential pair `admin:supersecret` was leaked and is available in the dark web! The credential as since been rotated everywhere where this account was used: the cache and the database. However, we are unsure how this was leaked. Did an attacked get access to the cache or to the database accounts? We want to investigate whether the attacker retrieved these credentials directly from Vault.

Retrieve all audit logs that are responses and contain `admin` in the username and `supersecret` in the password.

> Note: that is why credentials should never be reused. When credentials are only used in one place, not only is the rotation easier, but it is also trivial to pinpoint which system was compromised during the leak.

In [ ]:
vault write -field=hash /sys/audit-hash/file input='...'

In [ ]:
vault write -field=hash /sys/audit-hash/file input='...'

In [ ]:
jq 'select(.response.... == "..." and .response... == "...")' /tmp/vault-audit.log

#### Which component retrieved the credential?

## Cleaning Up

At the end of each module, you should clean up your Vault instance. This is done by shutting it down and wiping its database to restore its state.

In [ ]:
kill $(cat /tmp/vault.pid)
rm /tmp/vault.log
rm /tmp/vault.pid
rm /tmp/vault-audit.log